# Computing the Multiplicity B-Cubed Clustering Metric

The b-cubed clustering metric is often used to measure clustering performance. It has been demonstrated to avoid the weaknesses of other metrics for most applicatoins. See https://www.researchgate.net/profile/Julio_Gonzalo/publication/225548032_Amigo_E_Gonzalo_J_Artiles_J_et_alA_comparison_of_extrinsic_clustering_evaluation_metrics_based_on_formal_constraints_Inform_Retriev_12461-486/links/0c96052138dbb99740000000/Amigo-E-Gonzalo-J-Artiles-J-et-alA-comparison-of-extrinsic-clustering-evaluation-metrics-based-on-formal-constraints-Inform-Retriev-12461-486.pdf?_sg%5B0%5D=BVp5-cu12hNz0BrB9Jstgk_d8LpdMbhdL5uOE_ftkdtRBmpwbh6KQbF01DKGca87wgEaOkfDRYEcBayX9x2IGQ.bGbIvD_Ohtd-TcUi3gNjRvHm2oLy-qhuy9sQ7SeZRyOuZSf1zpEXmmxi5VrYR-zEzwP5d_uPAEz9nw-w_4AJNw&_sg%5B1%5D=F19xYSI3G3On97KihqlbmdQ64aGdnFyck-A7SJ-YGKAxjNnlJhI4bE5ltjIYEUCM438VJOrZ4EniRMTDSYPQA7_7yVQgbQoyxFZUbmYeSbj5.bGbIvD_Ohtd-TcUi3gNjRvHm2oLy-qhuy9sQ7SeZRyOuZSf1zpEXmmxi5VrYR-zEzwP5d_uPAEz9nw-w_4AJNw&_iepl= for a review and comparions of various clustering metrics. The implementation shown here is derived from this paper.

One advantage of the b-cubed metric is that performance is reported in terms that are familiar (precision, recall and f-score) which carry intuitive meaning for scientists, engineers and developers.

The explanation for computing the b-cubed precision metric is simply stated: Compute the precision score for each item in the population - the b-cubed precision is the average of the item precision scores. Similarly for recall: Compute the recall score for each item in the population - the b-cubed recall is the average of the item recall scores.

The reference paper has several good examples and illustrations.


## Categories and Clusters

The standard terminology is used throughout. The term ***categories*** refers the true or reference group of items in a population. A **category list** refers to the list of population items grouped by their category. The term ***clusters*** refers to an inferred or computed grouping of population items. The b-cubed metric proposed to score how well the inferred grouping compares to the reference grouping, i.e. how well do the clusters conform to the categories.

Category and cluster labels are arbitrary. They are external to the clustering problem. When a grouping identifier is needed within the b-cubed computation, a category or cluster is assigned an integer in the range \[0..N-1\] where N is the number of categories or clusters.

Items in the population, on the other hand, require some kind of identifer. For simplicity, we assume that items have an integer identifier in the range \[0..N-1\] where N is the number of items in the population.

## Category and Cluster Lists

A category or cluster is defined by the items that belong to it and is represented by a list of item identifiers (integers). A list of categories or clusters represents an entire population. In the simple case, an item belongs to one and only one category or cluster. In multiplicity case, an must belong to at least one category or cluster but may belong to multiple categories or clusters.

In the code, a category cohort consists of a list of category lists and a cluster cohort consists of a list of cluster lists. Their representations are identical, i.e. a list of integer lists. The order of item identifiers within the lists and the order of the categories and clusters within the cohort is arbitrary ... only the item identifiers need to be consistent between the two cohorts.

As an example, a 14 item population has the following category and cluster cohorts:

In [18]:
import numpy as np

categories = [[0,1,2],[0,1,3,4],[5,6]]
clusters = [[0,1,2,3,4],[5,6]]

This population of 14 items consists of 5 categories of 5, 6, 1, 1, and 1 item(s). The inferred solution consists of 3 clusters of 4, 3, and 7 items. Again, the item and category/cluster ordering is arbitrary. It is only imporant the the item identifiers are consistant between categories and clusters.

## The Implementation

The implementation consists of three steps:

1. compute inverse mappings for the cluster and category cohorts
2. create intersection count matrices
3. compute precision and recall

### Step 1 - Computing the inverse mappings

To create an inverse mapping we take the category(cluster) to node mapping and create a node to category (cluster) mapping. 

In [25]:
#
#   we first have to find out the number of items in the population
#
max_item = -1
for cluster in clusters:
    for item in cluster:
        max_item = max(max_item, item)
n_items = max_item+1

cluster_inverse_map = [set() for _ in range(n_items)]
for i, cluster in enumerate(clusters):
    for item in cluster:
        cluster_inverse_map[item].add(i)

category_inverse_map = [set() for _ in range(n_items)]
for i, category in enumerate(categories):
    for item in category:
        category_inverse_map[item].add(i)

print('inverse cluster map:', cluster_inverse_map)
print('inverse category map:', category_inverse_map)

inverse cluster map: [{0}, {0}, {0}, {0}, {0}, {1}, {1}]
inverse category map: [{0, 1}, {0, 1}, {0}, {1}, {1}, {2}, {2}]


Note that the lists of categories (clusters) that a node belongs to is actually a *set* and not a list. This will be important in a later operation. Using sets instead of lists is not a restriction since an item can not belong to the same category of cluster more than one time.

We now have a list of category (cluster) sets, one set for each item in the popluation. The next step is to determine the number of categories (clusters) each item has in common with other items.

### Step 2 - Create the Intersection Count Matrices

In this step, the category (cluster) set for each item is compared to the sets of other items. The number of categories (clusters) they have in common is stored in an NxN matrix, where N is the number of items. The names of the matrices are given as C and L (cluster and category respectively) to stay in sync with the reference documents.

In [27]:
C = np.zeros((n_items,n_items), dtype=np.int32)
L = np.zeros((n_items,n_items), dtype=np.int32)
#
#   we only populate the upper triangle of the matrices for now
#
for e1 in range(n_items):
    for e2 in range(e1,n_items):
        C[e1,e2] = len(cluster_inverse_map[e1] & cluster_inverse_map[e2])
        L[e1,e2] = len(category_inverse_map[e1] & category_inverse_map[e2])
#
#   make the matrices symmetric since the intersection of (e1,e2) is the same as (e1,e2)
#
C = np.bitwise_or(C, C.T)
L = np.bitwise_or(L, L.T)

print('C:')
print(C,'\n')
print('L:')
print(L)

C:
[[1 1 1 1 1 0 0]
 [1 1 1 1 1 0 0]
 [1 1 1 1 1 0 0]
 [1 1 1 1 1 0 0]
 [1 1 1 1 1 0 0]
 [0 0 0 0 0 1 1]
 [0 0 0 0 0 1 1]] 

L:
[[2 2 1 1 1 0 0]
 [2 2 1 1 1 0 0]
 [1 1 1 0 0 0 0]
 [1 1 0 1 1 0 0]
 [1 1 0 1 1 0 0]
 [0 0 0 0 0 1 1]
 [0 0 0 0 0 1 1]]


### Step 3 - Compute Precision and Recall

Now that we have the intersection counts, we can proceed with computing precision and recall. This involves first computing the "Multiplicity Precision" for each item pair (e1, e2) in the population. The formula for Multiplicy Precision and Recall is given as

$$
Multiplicity Precision = \frac{Min(|C(e1) \cap C(e2)|, |L(e1) \cap L(e2)|)}{|C(e1) \cap C(e2)}
Multiplicity Recall = \frac{Min(|C(e1) \cap C(e2)|, |L(e1) \cap L(e2)|)}{|L(e1) \cap L(e2)}
$$

Note that the intersections have already been computed and stored in matrices $C$ and $L$.

The numerator for the precision and recall computation is simply the minimum of $C$ and $L$ and the same for both computations so we compute that once. 

In [21]:
#
#   We can safely ignore divide by zero exceptions and warnings since we replace those results with zeros
#
with np.errstate(divide='ignore', invalid='ignore'):
    c_l_min = np.minimum.reduce([C,L])  # find minimum between category and cluster intersections
    p = np.nan_to_num(c_l_min/C)        # compute multi-precision for (e1,e2) pairs
    r = np.nan_to_num(c_l_min/L)        # compute multi-recall for (e1,e2) pairs


The rest of the computation consists of

- finding the average pair precision/recall score for each item; when taking these averages, it's important to only consider non-zero values, i.e. ony those elements where the category/cluster intersections are zero
- finding the average of the item averages

In [22]:
p_e_prime_ave = np.sum(p, axis=1)/np.sum(C != 0, axis=1)    # compute multi-precision e2 averages for each e1 where C is non-zero
p = np.average(p_e_prime_ave)                               # precision is the average of all e2 averages

r_e_prime_ave = np.sum(r, axis=1)/np.sum(L != 0, axis=1)    # compute multi-recall e2 averages for each e1 where L is non-zero
r = np.average(r_e_prime_ave)                               # recall is the average of all e2 averages


### Computing F-Score
The unweighted F-Score is computed from precision, P, and recall R by

$$
f score = 2\frac{RP}{R+P}
$$

In [23]:
f_score = 2.0*r*p/(r+p)

print(f'precision: {p:5.4f}\trecall: {r:5.4f}\tf_score: {f_score:5.4f}')

precision: 0.8857	recall: 0.9429	f_score: 0.9134
